## フェーズ1: 盤石な基盤モデルの構築 - モデルアーキテクチャの近代化（転移学習） (`feature/transfer_learning`)

データパイプラインとシンプルなCNNモデルの動作確認、およびK-Fold交差検証による評価基盤が確立されたため、次にモデルの性能を飛躍的に向上させるため、事前学習済みモデルを用いた転移学習を導入しました。

### やったこと

-   **`src/model.py` の改修**:
    -   `torchvision.models` を利用し、ImageNetで事前学習済みの`ConvNeXt_Tiny`モデル (`convnext_tiny`) を導入しました。
    -   `ConvNeXt_Tiny` の最終分類層を、本プロジェクトの10クラス分類に合わせて変更しました。
    -   動作確認用の`SimpleCNN`クラスは削除し、より汎用的な`HumanCharacterClassifier`クラスとして実装しました。
    -   将来的に他の事前学習済みモデルも容易に切り替えられるよう、`model_name`引数を追加しました。

-   **`src/train.py` の改修**:
    -   モデルの初期化部分を`SimpleCNN`から`HumanCharacterClassifier`へ変更しました。
    -   `MODEL_NAME`というグローバル変数を追加し、使用する転移学習モデルを簡単に指定できるようにしました。

### 動作確認

構築した転移学習モデルと学習スクリプトは、`src/train.py` を直接実行することで動作確認が可能です。

```bash
# srcディレクトリ内から実行
python3 train.py
```
実行すると、初回起動時にImageNetの事前学習済み重みが自動でダウンロードされ、その後K-Fold交差検証が開始されます。

出力例:
```
認識対象クラス数: 10
全データ数: 2501
Using device: cuda
Using model: convnext_tiny

--- Fold 1/5 ---
  訓練データ数 (Fold 1): 2000
  検証データ数 (Fold 1): 501
Downloading: "[https://download.pytorch.org/models/convnext_tiny-983f1562.pth](https://download.pytorch.org/models/convnext_tiny-983f1562.pth)" to /home/temmie0232/.cache/torch/hub/checkpoints/convnext_tiny-983f1562.pth
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 109M/109M [00:03<00:00, 31.2MB/s]
Epoch 1/10 (Train): 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 63/63 [01:02<00:00,  1.01it/s]
Epoch 1/10 (Validation): 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16/16 [00:07<00:00,  2.06it/s]
Epoch 1 | Train Loss: 1.3356 | Val Loss: 0.5403 | Val Accuracy: 80.04%
Epoch 2/10 (Train): 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 63/63 [00:57<00:00,  1.10it/s]
Epoch 2/10 (Validation): 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16/16 [00:07<00:00,  2.11it/s]
Epoch 2 | Train Loss: 0.2103 | Val Loss: 0.1429 | Val Accuracy: 97.41%
...
```
上記のように、学習の初期段階で既に高い検証精度（例: 97.41%）を達成しており、転移学習の導入が効果的であることが確認できました。これは、シンプルなCNNと比較して大幅な性能向上が見込めることを示唆しています。